In [ ]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define Project Folder
FOLDERNAME = 'Colab\ Notebooks/SC201L16'

%cd drive/MyDrive/$FOLDERNAME

In [ ]:
# Define device
import torch
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print('Device:', device)

In [16]:
# Load Existing Dataset
import torchvision.datasets as dset
import torchvision.transforms as T
transform = T.Compose([T.ToTensor(),T.RandomAutocontrast()])
train_data = dset.MNIST('./train',train=True,download=True,transform=transform)
val_data = dset.MNIST('./val',train=False,download=True,transform=transform)

In [ ]:
# Check Data Dimension
print(train_data)

In [19]:

train_data[0][0].shape ## print data

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./train
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               RandomAutocontrast(p=0.5)
           )


torch.Size([1, 28, 28])

In [ ]:
num_train = len(train_data)
num_val = len(val_data)
print('Number of training:', num_train)
print('Number of validation:', num_val)

In [20]:
# Build Model
import torch.nn as nn
model = None
model = nn.Sequential(
    # N x 1 x 28 x 28

    nn.Conv2d(in_channels=1,out_channels=64,kernel_size=3,padding=1,stride =1),
    nn.BatchNorm2d(num_features=64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2,stride=2),
    # 16 x 64 x 14 x 14

    nn.Flatten(),
    # 10 neurons(0-9)
    nn.Linear(64*14*14,10)
)

In [21]:
# Move model to GPU
# -> GPU
model = model.cuda()

In [22]:
# Create Mini-batches
from torch.utils.data import DataLoader
mini_trains = DataLoader(train_data, batch_size=128, shuffle=True)
mini_vals = DataLoader(val_data, batch_size=128, shuffle=True)

In [23]:
# Training Procedure
def train(num_epoch, model, mini_trains, mini_vals, device, loss_function, optimizer):
  for epoch in range(num_epoch):
    num_iters = 0
    for x, y in mini_trains:
      model.train()
      x = x.to(device)
      y = y.to(device)
      scores = model(x)
      loss = loss_function(scores, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if num_iters % 10 == 0:
        evaluate_predictor(model, epoch, mini_vals, device)
      num_iters += 1

In [24]:
# Validating Procedure
def evaluate_predictor(model, epoch, mini_vals, device):
  model.eval()
  with torch.no_grad():
    acc_count = 0
    for x, y in mini_vals:
      x = x.to(device)
      y = y.to(device)
      scores=model(x)
      predictions=scores.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_count += acc
    print(f'Epoch[{epoch+1}] Acc: {acc_count/num_val}')

In [25]:
# Define loss function & optimizer
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [26]:
# Start training
train(1, model, mini_trains, mini_vals, device, loss_function, optimizer)

NameError: name 'num_val' is not defined